## Caesar Cipher

In [2]:
def caesar_encrypt(plaintext, key):
    ciphertext = ""
    for char in plaintext:
        if char.isalpha():
            shift = 65 if char.isupper() else 97
            ciphertext += chr((ord(char) - shift + key) % 26 + shift)
        else:
            ciphertext += char
    return ciphertext


def caesar_decrypt(ciphertext, key):
    return caesar_encrypt(ciphertext, -key)


# Example
text = "HELLO WORLD"
key = 3

enc = caesar_encrypt(text, key)
dec = caesar_decrypt(enc, key)

print("Encrypted:", enc)
print("Decrypted:", dec)


Encrypted: KHOOR ZRUOG
Decrypted: HELLO WORLD


## Playfair Cipher

In [4]:
def generate_playfair_matrix(key):
    key = key.upper().replace("J", "I")
    matrix = []
    used = set()

    for char in key:
        if char.isalpha() and char not in used:
            used.add(char)
            matrix.append(char)

    for char in "ABCDEFGHIKLMNOPQRSTUVWXYZ":
        if char not in used:
            used.add(char)
            matrix.append(char)

    return [matrix[i*5:(i+1)*5] for i in range(5)]


In [6]:
def prepare_text(text):
    text = text.upper().replace("J", "I")
    result = ""

    i = 0
    while i < len(text):
        a = text[i]
        if not a.isalpha():
            i += 1
            continue

        if i+1 < len(text):
            b = text[i+1]
            if not b.isalpha():
                i += 1
                continue
        else:
            b = "X"

        if a == b:
            result += a + "X"
            i += 1  
        else:
            result += a + b
            i += 2

    if len(result) % 2 != 0:
        result += "X"

    return result


In [7]:
def find_position(matrix, char):
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                return i, j


In [9]:
def playfair_encrypt(plaintext, key):
    matrix = generate_playfair_matrix(key)
    text = prepare_text(plaintext)
    ciphertext = ""

    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2:
            ciphertext += matrix[r1][(c1+1)%5]
            ciphertext += matrix[r2][(c2+1)%5]
        elif c1 == c2:
            ciphertext += matrix[(r1+1)%5][c1]
            ciphertext += matrix[(r2+1)%5][c2]
        else:
            ciphertext += matrix[r1][c2]
            ciphertext += matrix[r2][c1]

    return ciphertext


def playfair_decrypt(ciphertext, key):
    matrix = generate_playfair_matrix(key)
    plaintext = ""

    for i in range(0, len(ciphertext), 2):
        a, b = ciphertext[i], ciphertext[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2:
            plaintext += matrix[r1][(c1-1)%5]
            plaintext += matrix[r2][(c2-1)%5]
        elif c1 == c2:
            plaintext += matrix[(r1-1)%5][c1]
            plaintext += matrix[(r2-1)%5][c2]
        else:
            plaintext += matrix[r1][c2]
            plaintext += matrix[r2][c1]

    return plaintext


In [12]:
enc = playfair_encrypt("HELLO", "MONARCHY")
dec = playfair_decrypt(enc, "MONARCHY")

print("Playfair Encrypted:", enc)
print("Playfair Decrypted:", dec)


Playfair Encrypted: CFSUPM
Playfair Decrypted: HELXLO


In [14]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.3 MB 1.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Hill Cipher

In [19]:
import numpy as np

def mod_inv(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None


In [20]:
def hill_encrypt(plaintext, key):
    plaintext = plaintext.upper().replace(" ", "")
    if len(plaintext) % 2 != 0:
        plaintext += "X"

    key = np.array(key)
    ciphertext = ""

    for i in range(0, len(plaintext), 2):
        pair = [ord(plaintext[i]) - 65, ord(plaintext[i+1]) - 65]
        vec = np.dot(key, pair) % 26
        ciphertext += chr(vec[0] + 65) + chr(vec[1] + 65)

    return ciphertext


In [21]:
def hill_decrypt(ciphertext, key):
    key = np.array(key)
    det = int(np.round(np.linalg.det(key))) % 26

    inv_det = mod_inv(det, 26)
    if inv_det is None:
        raise ValueError("Key matrix is not invertible modulo 26")

    adj = np.array([[key[1][1], -key[0][1]],
                    [-key[1][0], key[0][0]]])

    inv_key = (inv_det * adj) % 26

    plaintext = ""
    for i in range(0, len(ciphertext), 2):
        pair = [ord(ciphertext[i]) - 65, ord(ciphertext[i+1]) - 65]
        vec = np.dot(inv_key, pair) % 26
        plaintext += chr(vec[0] + 65) + chr(vec[1] + 65)

    return plaintext


In [22]:
key = [[3, 3],
       [2, 5]]

enc = hill_encrypt("HELLO", key)
dec = hill_decrypt(enc, key)

print("Hill Encrypted:", enc)
print("Hill Decrypted:", dec)


Hill Encrypted: HIOZHN
Hill Decrypted: HELLOX
